In [1]:
%reload_ext autoreload
%autoreload 2

from utils import load_dataset, SpecialTokens
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
m = 10000
data, human_vocab, machine_vocab = load_dataset(m)

100%|██████████| 10000/10000 [00:00<00:00, 31052.43it/s]


In [3]:
human_vocab

{';': 0,
 '?': 1,
 ' ': 2,
 '.': 3,
 '/': 4,
 '0': 5,
 '1': 6,
 '2': 7,
 '3': 8,
 '4': 9,
 '5': 10,
 '6': 11,
 '7': 12,
 '8': 13,
 '9': 14,
 'a': 15,
 'b': 16,
 'c': 17,
 'd': 18,
 'e': 19,
 'f': 20,
 'g': 21,
 'h': 22,
 'i': 23,
 'j': 24,
 'l': 25,
 'm': 26,
 'n': 27,
 'o': 28,
 'p': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'y': 36}

In [4]:
machine_vocab

{';': 0,
 '>': 1,
 '<': 2,
 '-': 3,
 '0': 4,
 '1': 5,
 '2': 6,
 '3': 7,
 '4': 8,
 '5': 9,
 '6': 10,
 '7': 11,
 '8': 12,
 '9': 13}

In [5]:
data[:20]

[('8/8/71', '>1971-08-08<'),
 ('3/3/81', '>1981-03-03<'),
 ('4/8/96', '>1996-04-08<'),
 ('4/9/78', '>1978-04-09<'),
 ('6/5/73', '>1973-06-05<'),
 ('5/2/89', '>1989-05-02<'),
 ('2/1/98', '>1998-02-01<'),
 ('8/5/79', '>1979-08-05<'),
 ('6/7/18', '>2018-06-07<'),
 ('6/8/89', '>1989-06-08<'),
 ('6/3/18', '>2018-06-03<'),
 ('2/7/21', '>2021-02-07<'),
 ('5/5/20', '>2020-05-05<'),
 ('8/6/88', '>1988-08-06<'),
 ('1/2/17', '>2017-01-02<'),
 ('7/6/99', '>1999-07-06<'),
 ('9/7/00', '>2000-09-07<'),
 ('3/8/78', '>1978-03-08<'),
 ('5/8/96', '>1996-05-08<'),
 ('5/3/19', '>2019-05-03<')]

In [6]:
class Lang:
    def _get_char(self, ind):
        if isinstance(ind, torch.Tensor):
            return self.inv_vocab[ind.item()]
        else:
            return self.inv_vocab[ind]

    def __init__(self, vocab: dict):
        self.vocab = vocab
        self.inv_vocab = {v:k for k,v in vocab.items()}
        self.vocab_size = len(vocab)

    def str_to_ind(self, str):
        return [self.vocab[c] for c in str]
    
    def ind_to_str(self, ind):
        return ''.join([self._get_char(i) for i in ind])

In [7]:
test = Lang(human_vocab)
date = data[0][0]
print(date)
translated_date = test.str_to_ind(date)
print(translated_date)
reversed_translation = test.ind_to_str(translated_date)
print(reversed_translation)

8/8/71
[13, 4, 13, 4, 12, 6]
8/8/71


In [8]:
class TranslationTrainingDataset(Dataset):
    def __init__(self, data, input_vocab, output_vocab):
        self.input_lang = Lang(input_vocab)
        self.target_lang = Lang(output_vocab)

        self.data = data

        self.encoder_inputs = [self.input_lang.str_to_ind(input_sent) for input_sent, _ in self.data]

        targets = [self.target_lang.str_to_ind(target_sent) for _, target_sent in self.data]
        self.decoder_inputs = [target[:-1] for target in targets]
        self.decoder_targets = [target[1:] for target in targets]

    def __getitem__(self, index):
        return self.encoder_inputs[index], self.decoder_inputs[index], self.decoder_targets[index]
    
    def __len__(self):
        return len(self.encoder_inputs)


In [9]:
dataset = TranslationTrainingDataset(data, human_vocab, machine_vocab)

In [10]:
x,y,z = dataset[0]
print(x, dataset.input_lang.ind_to_str(x))
print(y, dataset.target_lang.ind_to_str(y))
print(z, dataset.target_lang.ind_to_str(z))

[13, 4, 13, 4, 12, 6] 8/8/71
[1, 5, 13, 11, 5, 3, 4, 12, 3, 4, 12] >1971-08-08
[5, 13, 11, 5, 3, 4, 12, 3, 4, 12, 2] 1971-08-08<


In [12]:
def collate_batch(data):
    batch = []
    for i in range(len(data[0])):
        batch_data = [torch.tensor(item[i], dtype=torch.int64) for item in data]
        batch_data = nn.utils.rnn.pad_sequence(batch_data, batch_first=True)
        batch.append(batch_data)


    return tuple(batch)

In [13]:
loader = DataLoader(dataset=dataset, collate_fn=collate_batch, batch_size = 2, num_workers = 8)

In [24]:
class EncoderGRU(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1, bidirectional=False):
        super(EncoderGRU, self).__init__()
        self.D = 2 if bidirectional else 1
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.gru = nn.GRU(
            vocab_size,
            hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            batch_first=True,
        )

    def forward(self, x, hidden=None):
        if hidden == None:
            hidden = self.init_hidden(x.shape[0]).to(x.device)

        one_hot = F.one_hot(x, num_classes=self.vocab_size).float()

        output, hidden = self.gru(one_hot, hidden)

        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(
            self.D * self.gru.num_layers,
            batch_size,
            self.hidden_size,
            dtype=torch.float32,
        )

In [25]:
x_enc_batch,  x_dec_batch, y_batch = next(iter(loader))
print(x_enc_batch)
print(x_dec_batch)
print(y_batch)
encoder = EncoderGRU(len(human_vocab), hidden_size=32, num_layers=1, bidirectional=False)
print(encoder)
enc_out, enc_hidden = encoder(x_enc_batch)
print(enc_out.shape, enc_hidden.shape)

tensor([[13,  4, 13,  4, 12,  6],
        [ 8,  4,  8,  4, 13,  6]])
tensor([[ 1,  5, 13, 11,  5,  3,  4, 12,  3,  4, 12],
        [ 1,  5, 13, 12,  5,  3,  4,  7,  3,  4,  7]])
tensor([[ 5, 13, 11,  5,  3,  4, 12,  3,  4, 12,  2],
        [ 5, 13, 12,  5,  3,  4,  7,  3,  4,  7,  2]])
EncoderGRU(
  (gru): GRU(37, 32, batch_first=True)
)
torch.Size([2, 6, 32]) torch.Size([1, 2, 32])


In [26]:
class DecoderGRU(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size

        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.gru = nn.GRU(
            vocab_size, hidden_size, num_layers=num_layers, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        if hidden == None:
            hidden = self.init_hidden(x.shape[0]).to(x.device)
        one_hot = F.one_hot(x, num_classes=self.vocab_size).float()
        output, hidden = self.gru(one_hot, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(
            self.gru.num_layers, batch_size, self.hidden_size, dtype=torch.float32
        )

In [27]:
decoder = DecoderGRU(len(machine_vocab), hidden_size=32, num_layers=1)
decoder

DecoderGRU(
  (gru): GRU(14, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=14, bias=True)
)

In [39]:
print("Decoder forward pass\n")

# Teacher forcing
print("Training with teacher forcing")
print(f"Input batch shape: {x_dec_batch.shape}")
dec_out, dec_hid = decoder(x_dec_batch, enc_hidden)
print(f"decoder output shape: {dec_out.shape}\ndecoder hn shape: {dec_hid.shape}")
# loss(dec_out, target)
print()
# Without teacher forcing
print("Training without teacher forcing")
dec_input = x_dec_batch[:,0:1]
print(f"Decoder 1st input shape: {dec_input.shape}")
dec_out, dec_hid = decoder(dec_input, enc_hidden)
print(f"decoder output shape: {dec_out.shape}\ndecoder hn shape: {dec_hid.shape}")
next_input = torch.argmax(dec_out, dim=-1)
print(f"decoder 2nd input shape: {next_input.shape}")


Decoder forward pass

Training with teacher forcing
Input batch shape: torch.Size([2, 11])
decoder output shape: torch.Size([2, 11, 14])
decoder hn shape: torch.Size([1, 2, 32])

Training without teacher forcing
Decoder 1st input shape: torch.Size([2, 1])
decoder output shape: torch.Size([2, 1, 14])
decoder hn shape: torch.Size([1, 2, 32])
decoder 2nd input shape: torch.Size([2, 1])


In [85]:
class Seq2sec(nn.Module):
    def __init__(self, encoder, decoder) -> None:
        super(Seq2sec, self).__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_input_batch, sos_index = 1, dec_input_batch = None, teacher_forcing = False, out_length = 100):
        encoder_output, encoder_hidden = self.encoder(enc_input_batch)
        batch_size = len(enc_input_batch)

        if teacher_forcing:
            decoder_output, _ = self.decoder(dec_input_batch, encoder_hidden)
            return decoder_output
        else:
            decoder_input = torch.zeros(batch_size, 1, dtype=torch.int64) + sos_index
            decoder_output = torch.empty(batch_size, out_length, self.decoder.vocab_size)

            hidden = encoder_hidden

            for i in range(out_length):
                decoder_output_i, hidden = self.decoder(decoder_input, hidden)
                decoder_output[:,i:i+1,:] = decoder_output_i
                decoder_input = torch.argmax(decoder_output_i, dim=-1)

            return decoder_output


In [86]:
model = Seq2sec(encoder, decoder)
model

Seq2sec(
  (encoder): EncoderGRU(
    (gru): GRU(37, 32, batch_first=True)
  )
  (decoder): DecoderGRU(
    (gru): GRU(14, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=14, bias=True)
  )
)

In [87]:
model(x_enc_batch).shape

torch.Size([2, 100, 14])

In [ ]:
model(x_enc_batch, dec_input_batch = x_dec_batch, teacher_forcing = True).shape